In [1]:
import requests
import json
import os
from dotenv import load_dotenv
import pandas as pd
import datetime
import time
import csv
import psycopg2
from sqlalchemy import create_engine
import schedule

In [2]:
def get_token():
    load_dotenv('./.env')
    email = os.environ.get("email")
    password = os.environ.get("password")
    client_id = os.environ.get("X-ClientId")
    pass_key = os.environ.get("passKey")
    url = "https://openapi.emtmadrid.es/v3/mobilitylabs/user/login/"
    headers = {"email": email, "password" : password}
    response = requests.get(url, headers=headers)
    return response.content

In [3]:
def update_estaciones(): #Esta función hace UPDATE de los datos que no cambian de las estaciones en la base de datos
    load_dotenv('./.env')
    token = os.environ.get("access_token")
    url = "https://openapi.emtmadrid.es/v3/transport/bicimad/stations/"
    headers = {"accessToken" : token}
    stations = requests.get(url, headers = headers).json()
    date_and_time = datetime.datetime.now()
    date_and_time_formated = date_and_time.strftime("%Y-%m-%d %H:%M:%S")
    date_and_time_formated2 = date_and_time.strftime("%Y%m%d%H%M%S")
    estaciones = pd.DataFrame(stations["data"])
    estaciones["last_updated"] = date_and_time_formated
    estaciones[["longitude", "latitude"]] = estaciones["geometry"].apply(lambda x: pd.Series(x["coordinates"]))
    estaciones = estaciones.drop(["activate", "virtualDelete", "tipo_estacionPBSC", "geofence", "activate", "geometry", "integrator", "virtual_bikes", "virtual_bikes_num", "geofenced_capacity", "bikesGo"], axis=1)
    estaciones['coordinates'] = list(zip(estaciones['longitude'], estaciones['latitude']))
    update_estaciones = estaciones["address", "code_district", "code_suburb", "id", "number", "total_bases", "last_updated", "longitude", "latitude", "coordinates"]
    update_estaciones.to_csv(f'../data_csv/estaciones_{date_and_time_formated2}.csv')
    engine = create_engine("postgresql+psycopg2://postgres:"+ os.environ.get('postgre') +"@localhost/bicimad_worker")
    update_estaciones.to_sql('estaciones', con=engine, if_exists='replace', index=False)
    return update_estaciones

In [4]:
def get_disponibilidad(): #Esta función hace append en la base de datos de aquellas columnas que sí cambian a lo largo del día
    load_dotenv('./.env')
    token = os.environ.get("access_token")
    url = "https://openapi.emtmadrid.es/v3/transport/bicimad/stations/"
    headers = {"accessToken" : token}
    stations = requests.get(url, headers = headers).json()
    date_and_time = datetime.datetime.now()
    date_and_time_formated = date_and_time.strftime("%Y-%m-%d %H:%M:%S")
    date_and_time_formated2 = date_and_time.strftime("%Y%m%d%H%M%S")
    estaciones = pd.DataFrame(stations["data"])
    estaciones["last_updated"] = date_and_time_formated
    estaciones[["longitude", "latitude"]] = estaciones["geometry"].apply(lambda x: pd.Series(x["coordinates"]))
    estaciones = estaciones.drop(["activate", "virtualDelete", "tipo_estacionPBSC", "geofence", "activate", "geometry", "integrator", "virtual_bikes", "virtual_bikes_num", "geofenced_capacity", "bikesGo"], axis=1)
    estaciones['coordinates'] = list(zip(estaciones['longitude'], estaciones['latitude']))
    disponibilidad = estaciones["dock_bickes", "free_bases", "id", "light", "no_available", "reservations_count", "last_updated"]
    disponibilidad.to_csv(f'../data_csv/disponibilidad_{date_and_time_formated2}.csv')
    engine = create_engine("postgresql+psycopg2://postgres:"+ os.environ.get('postgre') +"@localhost/bicimad_worker")
    update_estaciones.to_sql('disponibilidad', con=engine, if_exists='append', index=False)
    return update_estaciones

In [12]:
#Función para recopilar data en tiempo real sobre las estaciones de bicimad y poder almacenarla en una database
"""
def get_stations():
    load_dotenv('./.env')
    token = os.environ.get("access_token")
    url = "https://openapi.emtmadrid.es/v3/transport/bicimad/stations/"
    headers = {"accessToken" : token}
    stations = requests.get(url, headers = headers).json()
    date_and_time = datetime.datetime.now()
    date_and_time_formated = date_and_time.strftime("%Y-%m-%d %H:%M:%S")
    date_and_time_formated2 = date_and_time.strftime("%Y%m%d%H%M%S")
    stations_real_time = pd.DataFrame(stations["data"])
    stations_real_time["last_updated"] = date_and_time_formated
    stations_real_time[["longitude", "latitude"]] = stations_real_time["geometry"].apply(lambda x: pd.Series(x["coordinates"]))
    stations_real_time = stations_real_time.drop(["activate", "virtualDelete", "tipo_estacionPBSC", "geofence", "activate", "geometry", "integrator", "virtual_bikes", "virtual_bikes_num", "geofenced_capacity", "bikesGo"], axis=1)
    stations_real_time['coordinates'] = list(zip(stations_real_time['longitude'], stations_real_time['latitude']))
    stations_real_time.to_csv(f'../data_csv/stations_{date_and_time_formated2}.csv')
    engine = create_engine("postgresql+psycopg2://postgres:"+ os.environ.get('postgre') +"@localhost/bicimad_worker")
    stations_real_time.to_sql('stations', con=engine, if_exists='append', index=False)
    return stations_real_time
"""

In [4]:
get_stations()

,address,code_district,code_suburb,dock_bikes,free_bases,id,light,name,no_available,number,reservations_count,total_bases,date_time,longitude,latitude,coordinates
0,"Calle María de las Mercedes de Borbón, 214,",16,166,5,18,2365,0,"262 - María de las Mercedes de Borbón, 214",0,262,0,23,2024-02-16 14:34:09,-3.620114,40.492868,"(-3.620114, 40.4928683)"
1,"Calle María de las Mercedes de Borbón, 98,",16,166,5,18,2366,0,"611 - María de las Mercedes de Borbón, 98",0,611,0,23,2024-02-16 14:34:09,-3.612107,40.490908,"(-3.6121068531215528, 40.490908297996654)"
2,"237- calle Francisco Balseiro,Comunidad de Mad...",06,061,18,6,2362,1,237 - calle Francisco Balseiro,0,237,0,24,2024-02-16 14:34:09,-3.710274,40.451682,"(-3.7102736, 40.4516819)"
3,"para borrar,",16,166,0,0,1404,3,0 - para borrar,1,0,0,0,2024-02-16 14:34:09,-3.620180,40.492910,"(-3.62018, 40.49291)"
4,"Calle Miguel Moya nº 1,",01,015,14,13,1406,2,2 - Metro Callao,0,2,0,27,2024-02-16 14:34:09,-3.705690,40.420400,"(-3.70569, 40.4204)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
608,"Cristo del Gran Poder, 27,",21,214,10,13,2238,2,"561 - Cristo del Gran Poder, 27",0,561,0,23,2024-02-16 14:34:09,-3.585751,40.472048,"(-3.585751, 40.472048)"
609,"Calle Tomillar, 40,",21,211,20,7,2239,1,"562 - Tomillar, 40",0,562,0,27,2024-02-16 14:34:09,-3.584415,40.463099,"(-3.5844146, 40.4630993)"
610,"560 - Playa de Zarauz, 16,",21,214,12,11,2240,2,"560 - Playa de Zarauz, 16",0,560,0,23,2024-02-16 14:34:09,-3.590685,40.473140,"(-3.5906847, 40.4731404)"
611,"5568 - Avda. del Partenón, 6,",21,215,9,18,2241,2,"568 - Avda. del Partenón, 6",0,568,0,27,2024-02-16 14:34:09,-3.617834,40.463280,"(-3.6178336, 40.4632802)"


In [6]:
#Esto hace que se ejecute la función cada hora y se carguen los datos en mi base de datos
schedule.every().hour.do(get_disponibilidad)

while True:
    schedule.run_pending()
    time.sleep(1)

schedule.every().month.do(update_estaciones)

while True:
    schedule.run_pending()
    time.sleep(1)

KeyError: ('dock_bickes', 'free_bases', 'id', 'light', 'no_available', 'reservations_count', 'last_updated')